### 7/26/2018
#### Trouble shooting (fixed)
Trouble with multiple copies of Anaconda/Python
1. Trouble getting pillow to work with anaconda. Had multiple copies of python and anaconda/miniconda installed, best way to solve a problem if module is not working is to probably uninstall anaconda and start over.
2. Do NOT have multiple copies of same program, this can definitely mess up things
3. Working on figuring out how to augment data using built in modules in python
-Image data Generator

#### Have completed
1. In Matlab, created series of rotated reference images and translated moving images. ***I want to take rotated reference images, and generate batches of translated/padded images as moved images***
2. I also tried making XxYx2 .mat input training set to act like image with 2 channels

#### To do
Goal: Train neural network to register sets of image with contrast changes to original image assuming possibility of local transformations, global transformations, and intensity changes as a function of time

1. Toy models:<br>
**Global Translation only**<br>
a. binary output, want NN to tell me whether image has been translated or not and thats it. May make most sense to try standard data set first<br>
b. classification, want NN to tell me how much image has been translated based on binned motion<br>
c. classification, want NN to output map telling me how much downsampled pixels have translated (similar to block matching)<br>
d. repeat c, but with corrected image as output<br>
**Global Rotation and Translation**<br>
**Local translations**<br>
2. Need to collect MR abdomen images at different contrast intensities for training neural network to remove contrast, and then register images (schedule for next week?)
3. ***Most Pressing***: How to input a reference and moving image into neural network during training using keras? How to label training data?(I think the flow_from_directory class can do this automatically, but I want to learn more on how to actually code it manually

#### Relevant Links
1. image preprocessing page
[link](https://keras.io/preprocessing/image/)
2. rotation net, estimate rotation of image (should take a look at the way he builds up a batch for training versus built-in-modules in keras
[link](https://github.com/d4nst/RotNet) 

### Code comparison between building class on own for data augmentation vs. ImageDataGenerator


compare the below code to ImageDataGenerator
class RotNetDataGenerator(Iterator):

    def __init__(self, input, batch_size=64,
                 preprocess_func=None, shuffle=False):

        self.images = input
        self.batch_size = batch_size
        self.input_shape = self.images.shape[1:]
        self.preprocess_func = preprocess_func
        self.shuffle = shuffle
        # add dimension if the images are greyscale
        if len(self.input_shape) == 2:
            self.input_shape = self.input_shape + (1,)
        N = self.images.shape[0]

        super(RotNetDataGenerator, self).__init__(N, batch_size, shuffle, None)

    def next(self):
        with self.lock:
            # get input data index and size of the current batch
            index_array, _, current_batch_size = next(self.index_generator)

        # create array to hold the images
        batch_x = np.zeros((current_batch_size,) + self.input_shape, dtype='float32')
       # create array to hold the labels
        batch_y = np.zeros(current_batch_size, dtype='float32')

        # iterate through the current batch
        for i, j in enumerate(index_array):
            image = self.images[j]

            # get a random angle
            rotation_angle = np.random.randint(360)

            # rotate the image
            rotated_image = rotate(image, rotation_angle)

            # add dimension to account for the channels if the image is greyscale
            if rotated_image.ndim == 2:
                rotated_image = np.expand_dims(rotated_image, axis=2)

            # store the image and label in their corresponding batches
            batch_x[i] = rotated_image
            batch_y[i] = rotation_angle

        # convert the numerical labels to binary labels
        batch_y = to_categorical(batch_y, 360)

        # preprocess input images
        if self.preprocess_func:
            batch_x = self.preprocess_func(batch_x)

        return batch_x, batch_y
